<a href="https://colab.research.google.com/github/RIDDHI1624/Drug-Discovery/blob/main/Insulin_Receptor_Project/Template_Selection_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install MDAnalysis biopython matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/validation
%cd /content/validation

Mounted at /content/drive
/content/validation


In [17]:
import os

# Copy from Drive
!cp "/content/drive/MyDrive/Project_Allostery_Step17/alphafold_champion_equilibrated.pdb" . 2>/dev/null
!cp "/content/drive/MyDrive/NPT_checkpoint/alphafold_champion.pdb" . 2>/dev/null

# If original not found, try other locations
if not os.path.exists("alphafold_champion.pdb"):
    !cp "/content/drive/MyDrive/alphafold_champion.pdb" . 2>/dev/null

if not os.path.exists("alphafold_champion.pdb"):
    from google.colab import files
    print("Upload alphafold_champion.pdb")
    uploaded = files.upload()

!ls -la *.pdb

-rw------- 1 root root 2812775 Feb 26 11:00 alphafold_champion_equilibrated.pdb
-rw-r--r-- 1 root root  169214 Feb 26 10:47 alphafold_champion.pdb
-rw-r--r-- 1 root root  333439 Feb 26 10:51 equil_protein_only.pdb


In [18]:
import MDAnalysis as mda
import numpy as np

u_orig = mda.Universe("alphafold_champion.pdb")

# Extract protein-only from equilibrated (it contains water + ions)
u_equil_full = mda.Universe("alphafold_champion_equilibrated.pdb")
protein_equil = u_equil_full.select_atoms("protein")
protein_equil.write("equil_protein_only.pdb")
u_equil = mda.Universe("equil_protein_only.pdb")

print(f"Original:     {u_orig.atoms.n_atoms} atoms, {u_orig.residues.n_residues} residues")
print(f"Equilibrated: {u_equil.atoms.n_atoms} atoms, {u_equil.residues.n_residues} residues")

Original:     2085 atoms, 260 residues
Equilibrated: 4114 atoms, 259 residues


In [19]:
print("=== Finding DFG Motif ===\n")
residues = list(u_equil.residues)
DFG_PHE = None
CAT_LYS = None
AC_GLU = None

for i in range(len(residues) - 2):
    r1, r2, r3 = residues[i], residues[i+1], residues[i+2]
    if r1.resname == "ASP" and r2.resname == "PHE" and r3.resname == "GLY":
        print(f"DFG motif found: D{r1.resid}-F{r2.resid}-G{r3.resid}")
        DFG_PHE = r2.resid
        break

for i in range(len(residues) - 3):
    r1, r2, r3, r4 = residues[i], residues[i+1], residues[i+2], residues[i+3]
    if r1.resname == "VAL" and r2.resname == "ALA" and r3.resname == "VAL" and r4.resname == "LYS":
        print(f"VAVK motif: Catalytic Lysine = LYS {r4.resid}")
        CAT_LYS = r4.resid
        break

for i in range(len(residues) - 3):
    r1, r2, r3, r4 = residues[i], residues[i+1], residues[i+2], residues[i+3]
    if r1.resname == "PHE" and r2.resname == "LEU" and r3.resname == "ASN" and r4.resname == "GLU":
        print(f"FLNE motif: aC-helix Glutamate = GLU {r4.resid}")
        AC_GLU = r4.resid
        break

print(f"\nKey Residues: DFG-Phe={DFG_PHE}, Cat-Lys={CAT_LYS}, aC-Glu={AC_GLU}")

=== Finding DFG Motif ===

DFG motif found: D126-F127-G128
VAVK motif: Catalytic Lysine = LYS 7
FLNE motif: aC-helix Glutamate = GLU 24

Key Residues: DFG-Phe=127, Cat-Lys=7, aC-Glu=24


In [21]:
def measure_chi1(universe, resid):
    res = universe.select_atoms(f"resid {resid}")
    N = res.select_atoms("name N")[0].position
    CA = res.select_atoms("name CA")[0].position
    CB = res.select_atoms("name CB")[0].position
    CG = res.select_atoms("name CG")[0].position
    b1 = CA - N
    b2 = CB - CA
    b3 = CG - CB
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    m1 = np.cross(n1, b2 / np.linalg.norm(b2))
    x = np.dot(n1, n2)
    y = np.dot(m1, n2)
    angle = np.degrees(np.arctan2(y, x))
    return angle

chi1_orig = measure_chi1(u_orig, DFG_PHE)
chi1_equil = measure_chi1(u_equil, DFG_PHE)

# Use absolute value — sign depends on convention, PyMOL gave -85.9
abs_orig = abs(chi1_orig)
abs_equil = abs(chi1_equil)

print("=" * 55)
print("  TEST 1: DFG-out Verification")
print("=" * 55)
print(f"  Requirement:   |60 to 90| degrees (gauche)")
print(f"  Original:      {chi1_orig:.1f} deg (|{abs_orig:.1f}|)")
print(f"  Equilibrated:  {chi1_equil:.1f} deg (|{abs_equil:.1f}|)")
print(f"  PyMOL ref:     -85.9 deg")
dfg_pass = 60 <= abs_equil <= 90
print(f"\n  {'PASS — DFG-out HELD' if dfg_pass else 'FAIL'}")

  TEST 1: DFG-out Verification
  Requirement:   |60 to 90| degrees (gauche)
  Original:      85.8 deg (|85.8|)
  Equilibrated:  85.9 deg (|85.9|)
  PyMOL ref:     -85.9 deg

  PASS — DFG-out HELD


In [22]:
def measure_salt_bridge(universe, lys_resid, glu_resid):
    lys_nz = universe.select_atoms(f"resid {lys_resid} and name NZ")
    glu_oe = universe.select_atoms(f"resid {glu_resid} and (name OE1 or name OE2)")
    if len(glu_oe) == 0:
        glu_oe = universe.select_atoms(f"resid {glu_resid} and name CD")
    min_dist = min(np.linalg.norm(lys_nz.positions[0] - pos) for pos in glu_oe.positions)
    return min_dist

dist_orig = measure_salt_bridge(u_orig, CAT_LYS, AC_GLU)
dist_equil = measure_salt_bridge(u_equil, CAT_LYS, AC_GLU)

print("=" * 55)
print("  TEST 2: aC-helix Position (Salt Bridge Distance)")
print("=" * 55)
print(f"  Requirement:   > 6.0 Angstrom")
print(f"  Original:      {dist_orig:.1f} Angstrom")
print(f"  Equilibrated:  {dist_equil:.1f} Angstrom")
sb_pass = dist_equil > 6.0
print(f"\n  {'PASS — Salt bridge BROKEN, aC-helix OUT' if sb_pass else 'FAIL'}")

  TEST 2: aC-helix Position (Salt Bridge Distance)
  Requirement:   > 6.0 Angstrom
  Original:      10.9 Angstrom
  Equilibrated:  11.5 Angstrom

  PASS — Salt bridge BROKEN, aC-helix OUT


In [23]:
ca_orig = u_orig.select_atoms("name CA")
ca_equil = u_equil.select_atoms("name CA")
n = min(len(ca_orig), len(ca_equil))

pos_orig = ca_orig.positions[:n].copy()
pos_equil = ca_equil.positions[:n].copy()

center_orig = pos_orig.mean(axis=0)
center_equil = pos_equil.mean(axis=0)
pos_orig -= center_orig
pos_equil -= center_equil

H = pos_equil.T @ pos_orig
U, S, Vt = np.linalg.svd(H)
d = np.linalg.det(Vt.T @ U.T)
sign_matrix = np.diag([1, 1, d])
R = Vt.T @ sign_matrix @ U.T
pos_equil_aligned = (R @ pos_equil.T).T

diff = pos_orig - pos_equil_aligned
squared = diff ** 2
summed = np.sum(squared, axis=1)
mean_val = np.mean(summed)
rmsd_val = np.sqrt(mean_val)

print("=" * 55)
print("  TEST 3: Global Structural Stability (Backbone RMSD)")
print("=" * 55)
print(f"  Requirement:   < 2.5 Angstrom")
print(f"  RMSD (Ca):     {rmsd_val:.3f} Angstrom")
rmsd_pass = rmsd_val < 2.5
if rmsd_val < 1.0:
    print("\n  EXCELLENT — Structure barely moved")
elif rmsd_val < 2.5:
    print("\n  PASS — Stable")
else:
    print("\n  FAIL — Significant drift")

  TEST 3: Global Structural Stability (Backbone RMSD)
  Requirement:   < 2.5 Angstrom
  RMSD (Ca):     0.274 Angstrom

  EXCELLENT — Structure barely moved


In [24]:
print()
print("=" * 60)
print("  PHASE 1 — STEP 18: TEMPLATE SELECTION QC REPORT")
print("  Project Allostery — INSR Kinase Domain")
print("=" * 60)
print()

header = f"{'Test':<30} {'Requirement':<18} {'Result':<14} {'Status'}"
print(header)
print("-" * 75)

s1 = "PASS" if dfg_pass else "FAIL"
s2 = "PASS" if sb_pass else "FAIL"
s3 = "PASS" if rmsd_pass else "FAIL"

print(f"{'1. DFG chi1 Angle':<30} {'-60 to -90 deg':<18} {chi1_equil:.1f} deg      {s1}")
print(f"{'2. Salt Bridge (aC-out)':<30} {'> 6.0 A':<18} {dist_equil:.1f} A         {s2}")
print(f"{'3. Backbone RMSD':<30} {'< 2.5 A':<18} {rmsd_val:.3f} A       {s3}")
print("-" * 75)

all_pass = dfg_pass and sb_pass and rmsd_pass

if all_pass:
    print("\nALL TESTS PASSED — DFG-out/aC-out CONFIRMED STABLE AT 310K")
    print("\nSYNTHETIC HOLO-TEMPLATE: alphafold_champion_equilibrated.pdb")
    print("\nValidated through:")
    print("  - AI prediction (AlphaFold 3)")
    print("  - Energy minimization (GROMACS)")
    print("  - 100ps NVT equilibration (310K)")
    print("  - 1ns NPT equilibration (310K, 1 bar)")
    print("  - Post-simulation conformational QC")
    print("\nPHASE 1 COMPLETE — Proceed to Phase 2: LiteFold Rosalind")
else:
    print("\nSOME TESTS FAILED")
    print("Recommendation: Use alphafold_champion_minimized.pdb instead")


  PHASE 1 — STEP 18: TEMPLATE SELECTION QC REPORT
  Project Allostery — INSR Kinase Domain

Test                           Requirement        Result         Status
---------------------------------------------------------------------------
1. DFG chi1 Angle              -60 to -90 deg     85.9 deg      PASS
2. Salt Bridge (aC-out)        > 6.0 A            11.5 A         PASS
3. Backbone RMSD               < 2.5 A            0.274 A       PASS
---------------------------------------------------------------------------

ALL TESTS PASSED — DFG-out/aC-out CONFIRMED STABLE AT 310K

SYNTHETIC HOLO-TEMPLATE: alphafold_champion_equilibrated.pdb

Validated through:
  - AI prediction (AlphaFold 3)
  - Energy minimization (GROMACS)
  - 100ps NVT equilibration (310K)
  - 1ns NPT equilibration (310K, 1 bar)
  - Post-simulation conformational QC

PHASE 1 COMPLETE — Proceed to Phase 2: LiteFold Rosalind


In [27]:
import json
from datetime import datetime

qc_report = {
    "project": "Project Allostery",
    "phase": "Phase 1 — Step 18: Template Selection",
    "date": datetime.now().strftime("%Y-%m-%d %H:%M"),
    "tests": {
        "DFG_chi1_angle": {"requirement": "-60 to -90 deg", "result": float(chi1_equil), "pass": True if dfg_pass else False},
        "salt_bridge": {"requirement": "> 6.0 A", "result": float(dist_equil), "pass": True if sb_pass else False},
        "backbone_RMSD": {"requirement": "< 2.5 A", "result": float(rmsd_val), "pass": True if rmsd_pass else False}
    },
    "all_passed": True if all_pass else False,
    "selected_template": "alphafold_champion_equilibrated.pdb"
}

with open("template_selection_qc.json", "w") as f:
    json.dump(qc_report, f, indent=2)

!cp alphafold_champion_equilibrated.pdb /content/drive/MyDrive/
!cp template_selection_qc.json /content/drive/MyDrive/

from google.colab import files
files.download("template_selection_qc.json")

print("\nPHASE 1 COMPLETE")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


PHASE 1 COMPLETE
